In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import threading
import time
from urllib.parse import urlsplit

In [2]:
service = Service(executable_path=r'C:\Program Files\Google\Chrome\Application\chromedriver')
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches",["ignore-certificate-errors"])
options.add_argument('--disable-gpu')
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")

# To make pages load faster by excluding some resources.
options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.xhr': 2,
                                                 'profile.managed_default_content_settings.preflight': 2,
                                                 'profile.managed_default_content_settings.images': 2,
                                                'profile.managed_default_content_settings.font': 2,
                                                'profile.managed_default_content_settings.css': 2})

driver = webdriver.Chrome(service=service, options=options)

In [3]:
def enter_url():
    url = input('Enter URL: ')

    # URL validation
    if not url.startswith("https"):
        url = 'https://' + url
    elif not url.startswith("http"):
        url = 'http://' + url
    else:
        pass
    
    return url

In [26]:
url = enter_url()

def base_url(url):
    parts = urlsplit(url)
    base = '{0.netloc}'.format(parts)
    temp = base.split('.')
    remove_this = temp[0] + '.'
    base = base.replace(remove_this, '')
    
    return base

domain_url = base_url(url)
print(domain_url)
links_to_scrape = []
links_to_scrape.append(url)
scraped_links = []
all_links = {}

Enter URL: www.google.com
google.com


In [27]:
def link_finder():
    
    url = links_to_scrape[0]
    if url == None:
        return None
    driver.get(url)

    # Give the required tag and/or class, etc.
    elems = driver.find_elements(by=By.TAG_NAME, value="a")
    links_found = []
    # Save the scraped links in list
    for elem in elems:
        link = elem.get_attribute("href")
        links_found.append(link)
        all_links[link] = 1
        
        
    scraped_links.append(url)
    links_to_scrape.remove(url)
    
    for link in links_found:
        link_base = base_url(link)
        if domain_url == link_base:
            if link in scraped_links:
                pass
            else:
                links_to_scrape.append(link)
                
    
            
    driver.quit()

In [28]:
N = 1   # Number of browsers to spawn
thread_list = list()

# Start test
for i in range(N):
    t = threading.Thread(name='Test {}'.format(i), target=link_finder)
    t.start()
    time.sleep(1)
    print(t.name + ' started!')
    thread_list.append(t)

# Wait for all threads to complete
for thread in thread_list:
    thread.join()

print('Test completed!')

Test 0 started!
Test completed!


In [29]:
print(len(all_links))

19


In [30]:
print(all_links)

{'https://mail.google.com/mail/&ogbl': 1, 'https://www.google.com/imghp?hl=en&ogbl': 1, 'https://www.google.com.pk/intl/en/about/products': 1, 'https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=https://www.google.com/&ec=GAZAmgQ': 1, 'https://www.google.com/#': 1, 'https://www.google.com/setprefs?sig=0_yRCZXqdrJkcxZBvkxL-AgQDiobY%3D&hl=ur&source=homepage&sa=X&ved=0ahUKEwjh4pKWgbyAAxUfTqQEHQyyAasQ2ZgBCBE': 1, 'https://www.google.com/setprefs?sig=0_yRCZXqdrJkcxZBvkxL-AgQDiobY%3D&hl=ps&source=homepage&sa=X&ved=0ahUKEwjh4pKWgbyAAxUfTqQEHQyyAasQ2ZgBCBI': 1, 'https://www.google.com/setprefs?sig=0_yRCZXqdrJkcxZBvkxL-AgQDiobY%3D&hl=sd&source=homepage&sa=X&ved=0ahUKEwjh4pKWgbyAAxUfTqQEHQyyAasQ2ZgBCBM': 1, 'https://about.google/?utm_source=google-PK&utm_medium=referral&utm_campaign=hp-footer&fg=1': 1, 'https://www.google.com/intl/en_pk/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpafooter&fg=1': 1, 'https://www.goog

In [31]:
print(scraped_links)

['https://www.google.com']


In [32]:
print(links_to_scrape)

['https://mail.google.com/mail/&ogbl', 'https://www.google.com/imghp?hl=en&ogbl', 'https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=https://www.google.com/&ec=GAZAmgQ', 'https://www.google.com/#', 'https://www.google.com/setprefs?sig=0_yRCZXqdrJkcxZBvkxL-AgQDiobY%3D&hl=ur&source=homepage&sa=X&ved=0ahUKEwjh4pKWgbyAAxUfTqQEHQyyAasQ2ZgBCBE', 'https://www.google.com/setprefs?sig=0_yRCZXqdrJkcxZBvkxL-AgQDiobY%3D&hl=ps&source=homepage&sa=X&ved=0ahUKEwjh4pKWgbyAAxUfTqQEHQyyAasQ2ZgBCBI', 'https://www.google.com/setprefs?sig=0_yRCZXqdrJkcxZBvkxL-AgQDiobY%3D&hl=sd&source=homepage&sa=X&ved=0ahUKEwjh4pKWgbyAAxUfTqQEHQyyAasQ2ZgBCBM', 'https://www.google.com/intl/en_pk/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpafooter&fg=1', 'https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1', 'https://policies.google.com/privacy?hl=e